In [3]:
# import os
# import rasterio
# from rasterio.enums import Resampling
# import numpy as np

# def weighted_overlay(output_raster_path, factor_rasters, weights, nodata_val=-9999):
#     """
#     Performs a weighted overlay on multiple factor rasters, ensuring they are
#     resampled to a common grid to prevent shape mismatch errors.

#     Args:
#         output_raster_path (str): Path to save the final suitability raster.
#         factor_rasters (dict): A dictionary where keys are factor names and values
#                                are the paths to the corresponding suitability raster files.
#         weights (dict): A dictionary where keys are factor names and values are
#                         their weights (should sum to 1.0).
#         nodata_val (int): The NoData value for the output raster.
#     """
#     print(f"\n--- Starting Weighted Overlay Analysis ---")

#     try:
#         # --- Step 1: Open the first raster to define the master grid ---
#         # All other rasters will be conformed to the shape and transform of this raster.
#         first_factor_name = list(factor_rasters.keys())[0]
#         master_raster_path = factor_rasters[first_factor_name]
        
#         if not os.path.exists(master_raster_path):
#             print(f"!!! Error: Master grid file not found at '{master_raster_path}'. Cannot proceed.")
#             return

#         with rasterio.open(master_raster_path) as src:
#             master_meta = src.meta.copy()
#             master_shape = src.shape
            
#             print(f"Master grid defined by '{os.path.basename(master_raster_path)}' with shape: {master_shape}")

#             # Initialize the final weighted sum array with zeros
#             weighted_sum = np.zeros(master_shape, dtype=np.float32)
#             # Create a mask to keep track of NoData pixels across all layers
#             final_nodata_mask = np.full(master_shape, False, dtype=bool)

#         # --- Step 2: Iterate through each factor raster ---
#         for factor_name, raster_path in factor_rasters.items():
#             if not os.path.exists(raster_path):
#                 print(f"!!! Error: Input file not found at '{raster_path}'. Skipping this factor.")
#                 continue

#             weight = weights[factor_name]
#             print(f"Processing factor: '{factor_name}' with weight: {weight:.2f}")

#             with rasterio.open(raster_path) as src:
#                 # Read the data, resampling it on-the-fly to match the master grid's shape.
#                 # This is the key step that solves the broadcasting error.
#                 factor_data = src.read(
#                     1,
#                     out_shape=master_shape,
#                     resampling=Resampling.bilinear # Use bilinear for continuous-like suitability scores
#                 ).astype(np.float32)

#                 input_nodata_val = src.nodata
                
#                 # Create a mask for NoData values AFTER resampling
#                 nodata_mask = (factor_data == input_nodata_val)
#                 final_nodata_mask = final_nodata_mask | nodata_mask
                
#                 # Set NoData pixels to 0 for the calculation so they don't affect the sum
#                 factor_data[nodata_mask] = 0
                
#                 # Apply the weight and add to the total weighted sum
#                 weighted_sum += (factor_data * weight)
        
#         # --- Step 3: Finalize the suitability map ---
#         # Where any input layer had NoData, set the final output to NoData
#         weighted_sum[final_nodata_mask] = nodata_val
        
#         # Update metadata for the final output file using the master grid's metadata
#         master_meta.update(
#             dtype=rasterio.float32, # The result is a float
#             count=1,
#             nodata=nodata_val,
#             compress='lzw'
#         )

#         print(f"\nWriting final suitability map to: {output_raster_path}")
#         with rasterio.open(output_raster_path, 'w', **master_meta) as dst:
#             dst.write(weighted_sum, 1)

#         print(f"\nSuccessfully created the final suitability map!")

#     except Exception as e:
#         print(f"An error occurred during the weighted overlay: {e}")

# # --- MAIN SCRIPT EXECUTION ---
# if __name__ == "__main__":
#     # --- USER: VERIFY YOUR INPUTS AND WEIGHTS HERE ---

#     # 1. Define the paths to your three reclassified suitability rasters
#     suitability_layers_dir = "C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA"
    
#     factor_raster_paths = {
#         "slope": os.path.join(suitability_layers_dir, "Slope_Suitability.tif"),
#         "distance": os.path.join(suitability_layers_dir, "Distance_Suitability.tif"),
#         "ndvi": os.path.join(suitability_layers_dir, "NDVI_Suitability.tif")
#     }

#     # 2. Define the weights for each factor. These MUST sum to 1.0.
#     factor_weights = {
#         "slope": 0.50,
#         "distance": 0.30,
#         "ndvi": 0.20
#     }
    
#     if not np.isclose(sum(factor_weights.values()), 1.0):
#         raise ValueError(f"Weights must sum to 1.0, but they sum to {sum(factor_weights.values())}")

#     # 3. Define the path for the final output map
#     final_output_path = "C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/Final_Dam_Suitability.tif"
    
#     # --- END OF USER SETTINGS ---

#     weighted_overlay(final_output_path, factor_raster_paths, factor_weights)



--- Starting Weighted Overlay Analysis ---
Master grid defined by 'Slope_Suitability.tif' with shape: (3601, 3601)
Processing factor: 'slope' with weight: 0.50
Processing factor: 'distance' with weight: 0.30
Processing factor: 'ndvi' with weight: 0.20

Writing final suitability map to: C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/DATA/Final_Dam_Suitability.tif

Successfully created the final suitability map!


In [3]:
import os
import rasterio
from rasterio.enums import Resampling
import numpy as np

def weighted_overlay(output_raster_path, factor_rasters, weights, nodata_val=-9999):
    """
    Performs a weighted overlay on multiple factor rasters, ensuring they are
    resampled to a common grid to prevent shape mismatch errors.

    Args:
        output_raster_path (str): Path to save the final suitability raster.
        factor_rasters (dict): A dictionary where keys are factor names and values
                               are the paths to the corresponding suitability raster files.
        weights (dict): A dictionary where keys are factor names and values are
                        their weights (should sum to 1.0).
        nodata_val (int): The NoData value for the output raster.
    """
    print(f"\n--- Starting Weighted Overlay Analysis ---")

    try:
        # --- Step 1: Open the first raster to define the master grid ---
        # All other rasters will be conformed to the shape and transform of this raster.
        first_factor_name = list(factor_rasters.keys())[0]
        master_raster_path = factor_rasters[first_factor_name]
        
        if not os.path.exists(master_raster_path):
            print(f"!!! Error: Master grid file not found at '{master_raster_path}'. Cannot proceed.")
            return

        with rasterio.open(master_raster_path) as src:
            master_meta = src.meta.copy()
            master_shape = src.shape
            
            print(f"Master grid defined by '{os.path.basename(master_raster_path)}' with shape: {master_shape}")

            # Initialize the final weighted sum array with zeros
            weighted_sum = np.zeros(master_shape, dtype=np.float32)
            # Create a mask to keep track of NoData pixels across all layers
            final_nodata_mask = np.full(master_shape, False, dtype=bool)

        # --- Step 2: Iterate through each factor raster ---
        for factor_name, raster_path in factor_rasters.items():
            if not os.path.exists(raster_path):
                print(f"!!! Error: Input file not found at '{raster_path}'. Skipping this factor.")
                continue

            weight = weights[factor_name]
            print(f"Processing factor: '{factor_name}' with weight: {weight:.2f}")

            with rasterio.open(raster_path) as src:
                # Read the data, resampling it on-the-fly to match the master grid's shape.
                # This is the key step that solves the broadcasting error.
                factor_data = src.read(
                    1,
                    out_shape=master_shape,
                    resampling=Resampling.bilinear # Use bilinear for continuous-like suitability scores
                ).astype(np.float32)

                input_nodata_val = src.nodata
                
                # Create a mask for NoData values AFTER resampling
                nodata_mask = (factor_data == input_nodata_val)
                final_nodata_mask = final_nodata_mask | nodata_mask
                
                # Set NoData pixels to 0 for the calculation so they don't affect the sum
                factor_data[nodata_mask] = 0
                
                # Apply the weight and add to the total weighted sum
                weighted_sum += (factor_data * weight)
        
        # --- Step 3: Finalize the suitability map ---
        # Where any input layer had NoData, set the final output to NoData
        weighted_sum[final_nodata_mask] = nodata_val
        
        # Update metadata for the final output file using the master grid's metadata
        master_meta.update(
            dtype=rasterio.float32, # The result is a float
            count=1,
            nodata=nodata_val,
            compress='lzw'
        )

        print(f"\nWriting final suitability map to: {output_raster_path}")
        with rasterio.open(output_raster_path, 'w', **master_meta) as dst:
            dst.write(weighted_sum, 1)

        print(f"\nSuccessfully created the final suitability map!")

    except Exception as e:
        print(f"An error occurred during the weighted overlay: {e}")

# --- MAIN SCRIPT EXECUTION ---
if __name__ == "__main__":
    # --- USER: VERIFY YOUR INPUTS AND WEIGHTS HERE ---

    # 1. Define the path to your Suitability_Layers folder
    # This path should point to your Dam_Suitability_Analysis/Data/Processed_Rasters/Suitability_Layers/
    suitability_layers_dir = "C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/Data/Processed_Rasters"
    
    factor_raster_paths = {
        "slope": os.path.join(suitability_layers_dir, "Slope_Suitability.tif"),
        "distance": os.path.join(suitability_layers_dir, "Distance_Suitability.tif"),
        "ndvi": os.path.join(suitability_layers_dir, "NDVI_Suitability.tif"),
        "depression": os.path.join(suitability_layers_dir, "Depression_Suitability.tif"),
        "flow_accumulation": os.path.join(suitability_layers_dir, "Flow_Accumulation_Suitability.tif") # ADDED THIS LINE
    }

    # 2. Define the weights for each factor. These MUST sum to 1.0.
    # Adjust these weights based on the importance you assign to each factor.
    # Example weights (ensure they sum to 1.0):
    factor_weights = {
        "slope": 0.20,      # Reduced from 0.50
        "distance": 0.15,   # Reduced from 0.30
        "ndvi": 0.05,       # Reduced from 0.20
        "depression": 0.30,  # Reduced from 0.40
        "flow_accumulation": 0.30 # New, significant weight
    }
    
    if not np.isclose(sum(factor_weights.values()), 1.0):
        raise ValueError(f"Weights must sum to 1.0, but they sum to {sum(factor_weights.values())}")

    # 3. Define the path for the final output map
    # This path will save the final output in your Processed_Rasters folder
    final_output_path = "C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/Data/Processed_Rasters/Final_Dam_Suitability.tif"
    
    # --- END OF USER SETTINGS ---

    weighted_overlay(final_output_path, factor_raster_paths, factor_weights)


--- Starting Weighted Overlay Analysis ---
Master grid defined by 'Slope_Suitability.tif' with shape: (3601, 3601)
Processing factor: 'slope' with weight: 0.20
Processing factor: 'distance' with weight: 0.15
Processing factor: 'ndvi' with weight: 0.05
Processing factor: 'depression' with weight: 0.30
Processing factor: 'flow_accumulation' with weight: 0.30

Writing final suitability map to: C:/Users/Lenovo/Documents/Dam_Suitability_Analysis/Data/Processed_Rasters/Final_Dam_Suitability.tif

Successfully created the final suitability map!
